In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

from matplotlib.pyplot import imshow

In [20]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\flower-color-images\flower_images\flower_images"):
            for file in f:
                if '.png' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = np.array(img)
            x = x[...,:3]
            if(x.shape == (128,128,3)):
                data.append(x)

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),128,128,3)
        return x_train

In [24]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(512 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 512)))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("sigmoid"))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("tanh"))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("sigmoid"))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("sigmoid"))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 256
        gen_imgs = gen_imgs.astype(int)

        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        self.generator.save("generator.h5")

In [26]:
dcgan = DCGAN()
dcgan.train(epochs=5001, batch_size=10, save_interval=100)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 33, 33, 64)        0         
_________________________________________________________________
batch_normalization_52 (Batc (None, 33, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 33, 33, 64)      

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 2.339221, acc.: 10.00%] [G loss: 0.394427]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.031236, acc.: 45.00%] [G loss: 2.046086]
2 [D loss: 0.732229, acc.: 70.00%] [G loss: 1.561848]
3 [D loss: 1.087539, acc.: 30.00%] [G loss: 0.975666]
4 [D loss: 0.928625, acc.: 30.00%] [G loss: 0.916241]
5 [D loss: 0.430822, acc.: 90.00%] [G loss: 0.711573]
6 [D loss: 0.650318, acc.: 65.00%] [G loss: 1.259095]
7 [D loss: 0.503717, acc.: 80.00%] [G loss: 0.472503]
8 [D loss: 0.327635, acc.: 90.00%] [G loss: 0.818698]
9 [D loss: 1.556762, acc.: 35.00%] [G loss: 1.199169]
10 [D loss: 0.676667, acc.: 70.00%] [G loss: 0.330321]
11 [D loss: 0.364184, acc.: 85.00%] [G loss: 0.061403]
12 [D loss: 0.117280, acc.: 95.00%] [G loss: 0.028229]
13 [D loss: 0.037321, acc.: 100.00%] [G loss: 0.034149]
14 [D loss: 0.018238, acc.: 100.00%] [G loss: 0.000640]
15 [D loss: 0.010208, acc.: 100.00%] [G loss: 0.055792]
16 [D loss: 0.618115, acc.: 85.00%] [G loss: 0.012296]
17 [D loss: 1.359899, acc.: 75.00%] [G loss: 0.156236]
18 [D loss: 1.004379, acc.: 65.00%] [G loss: 1.079818]
19 [D loss: 1.24

...

4337 [D loss: 0.739826, acc.: 75.00%] [G loss: 3.977286]
4338 [D loss: 0.116409, acc.: 95.00%] [G loss: 5.151958]
4339 [D loss: 0.354180, acc.: 90.00%] [G loss: 3.201646]
4340 [D loss: 0.257404, acc.: 90.00%] [G loss: 3.372189]
4341 [D loss: 0.461866, acc.: 70.00%] [G loss: 2.299600]
4342 [D loss: 0.327039, acc.: 80.00%] [G loss: 2.322543]
4343 [D loss: 0.392683, acc.: 80.00%] [G loss: 1.405221]
4344 [D loss: 0.164534, acc.: 90.00%] [G loss: 2.014092]
4345 [D loss: 0.145489, acc.: 95.00%] [G loss: 1.928145]
4346 [D loss: 0.322085, acc.: 90.00%] [G loss: 1.042061]
4347 [D loss: 1.144569, acc.: 45.00%] [G loss: 4.250924]
4348 [D loss: 0.701736, acc.: 80.00%] [G loss: 2.941964]
4349 [D loss: 0.076270, acc.: 100.00%] [G loss: 3.494189]
4350 [D loss: 0.665851, acc.: 55.00%] [G loss: 3.915939]
4351 [D loss: 0.070219, acc.: 95.00%] [G loss: 5.801507]
4352 [D loss: 0.300786, acc.: 85.00%] [G loss: 3.777248]
4353 [D loss: 0.393792, acc.: 80.00%] [G loss: 3.426539]
4354 [D loss: 0.495426, acc.: 

4481 [D loss: 0.819315, acc.: 60.00%] [G loss: 3.183947]
4482 [D loss: 0.270786, acc.: 85.00%] [G loss: 3.034613]
4483 [D loss: 0.104940, acc.: 100.00%] [G loss: 2.364311]
4484 [D loss: 0.268094, acc.: 80.00%] [G loss: 3.226315]
4485 [D loss: 0.201447, acc.: 85.00%] [G loss: 2.409058]
4486 [D loss: 0.367324, acc.: 85.00%] [G loss: 4.004609]
4487 [D loss: 0.725390, acc.: 75.00%] [G loss: 3.957101]
4488 [D loss: 0.338335, acc.: 80.00%] [G loss: 4.697738]
4489 [D loss: 0.322696, acc.: 85.00%] [G loss: 2.509175]
4490 [D loss: 0.363203, acc.: 75.00%] [G loss: 3.047668]
4491 [D loss: 0.230485, acc.: 80.00%] [G loss: 3.526048]
4492 [D loss: 0.483622, acc.: 75.00%] [G loss: 2.018716]
4493 [D loss: 0.381485, acc.: 90.00%] [G loss: 2.866014]
4494 [D loss: 0.978546, acc.: 60.00%] [G loss: 4.113630]
4495 [D loss: 0.036152, acc.: 100.00%] [G loss: 2.179874]
4496 [D loss: 0.299314, acc.: 95.00%] [G loss: 3.914121]
4497 [D loss: 0.163512, acc.: 95.00%] [G loss: 4.570979]
4498 [D loss: 0.188854, acc.:

4625 [D loss: 0.163957, acc.: 95.00%] [G loss: 2.100866]
4626 [D loss: 0.608639, acc.: 75.00%] [G loss: 4.480859]
4627 [D loss: 0.032758, acc.: 100.00%] [G loss: 2.191510]
4628 [D loss: 0.124135, acc.: 95.00%] [G loss: 1.206897]
4629 [D loss: 0.122969, acc.: 100.00%] [G loss: 2.593375]
4630 [D loss: 0.280231, acc.: 85.00%] [G loss: 0.938724]
4631 [D loss: 0.174806, acc.: 95.00%] [G loss: 2.286323]
4632 [D loss: 0.164186, acc.: 95.00%] [G loss: 1.393597]
4633 [D loss: 0.505200, acc.: 85.00%] [G loss: 1.843623]
4634 [D loss: 0.172072, acc.: 90.00%] [G loss: 3.373908]
4635 [D loss: 0.399667, acc.: 85.00%] [G loss: 2.920680]
4636 [D loss: 0.415201, acc.: 85.00%] [G loss: 2.968112]
4637 [D loss: 0.275913, acc.: 85.00%] [G loss: 1.923364]
4638 [D loss: 0.087132, acc.: 95.00%] [G loss: 2.173154]
4639 [D loss: 0.275362, acc.: 80.00%] [G loss: 1.180356]
4640 [D loss: 0.092408, acc.: 100.00%] [G loss: 1.913607]
4641 [D loss: 0.468224, acc.: 85.00%] [G loss: 2.258624]
4642 [D loss: 0.223162, acc.

4769 [D loss: 0.398435, acc.: 85.00%] [G loss: 1.552531]
4770 [D loss: 0.780133, acc.: 70.00%] [G loss: 4.670887]
4771 [D loss: 0.298124, acc.: 85.00%] [G loss: 5.710783]
4772 [D loss: 0.641594, acc.: 70.00%] [G loss: 3.180571]
4773 [D loss: 0.609814, acc.: 75.00%] [G loss: 2.728751]
4774 [D loss: 0.046417, acc.: 100.00%] [G loss: 2.940966]
4775 [D loss: 0.245778, acc.: 90.00%] [G loss: 2.691688]
4776 [D loss: 0.397022, acc.: 80.00%] [G loss: 1.935183]
4777 [D loss: 1.154433, acc.: 55.00%] [G loss: 3.248094]
4778 [D loss: 0.112993, acc.: 100.00%] [G loss: 3.042395]
4779 [D loss: 0.536609, acc.: 75.00%] [G loss: 1.653179]
4780 [D loss: 0.148883, acc.: 95.00%] [G loss: 1.991576]
4781 [D loss: 0.622062, acc.: 70.00%] [G loss: 4.239671]
4782 [D loss: 1.317713, acc.: 40.00%] [G loss: 4.083820]
4783 [D loss: 0.317056, acc.: 85.00%] [G loss: 4.819253]
4784 [D loss: 0.285915, acc.: 90.00%] [G loss: 2.893474]
4785 [D loss: 0.102602, acc.: 100.00%] [G loss: 3.318198]
4786 [D loss: 0.167419, acc.

4913 [D loss: 0.266458, acc.: 90.00%] [G loss: 2.521628]
4914 [D loss: 0.541952, acc.: 70.00%] [G loss: 3.934643]
4915 [D loss: 0.239475, acc.: 90.00%] [G loss: 4.412592]
4916 [D loss: 0.280322, acc.: 90.00%] [G loss: 3.687640]
4917 [D loss: 0.290575, acc.: 90.00%] [G loss: 2.879564]
4918 [D loss: 0.312307, acc.: 85.00%] [G loss: 5.644025]
4919 [D loss: 0.096705, acc.: 100.00%] [G loss: 5.428499]
4920 [D loss: 1.207188, acc.: 60.00%] [G loss: 3.975409]
4921 [D loss: 0.148436, acc.: 95.00%] [G loss: 3.269677]
4922 [D loss: 0.125775, acc.: 95.00%] [G loss: 3.482170]
4923 [D loss: 0.259001, acc.: 85.00%] [G loss: 3.611239]
4924 [D loss: 0.193618, acc.: 95.00%] [G loss: 3.619805]
4925 [D loss: 0.680024, acc.: 70.00%] [G loss: 4.308423]
4926 [D loss: 0.753223, acc.: 65.00%] [G loss: 4.609963]
4927 [D loss: 0.056902, acc.: 100.00%] [G loss: 3.152520]
4928 [D loss: 0.074038, acc.: 95.00%] [G loss: 2.302670]
4929 [D loss: 0.220609, acc.: 80.00%] [G loss: 1.805367]
4930 [D loss: 0.583618, acc.: